# Widgets Tests

Tests various widgets in package `github.com/janpfeifer/gonb/gonbui/widgets`.

In [1]:
%goflags --cover --covermode=set

%goflags=["--cover" "--covermode=set"]


In [2]:
!if [[ "${GONB_GIT_ROOT}" == "" ]] ; then \
    echo "Please set GONB_GIT_ROOT before runnig this notebook!" 1>&2 ; \
else \
    echo "ok" ; \
fi

ok


In [3]:
!*rm -f go.work && go work init && go work use . "${GONB_GIT_ROOT}"
%goworkfix

	- Added replace rule for module "github.com/janpfeifer/gonb" to local directory "/home/janpf/Projects/gonb".


### Buttons

Create a button and induce a click on it using Javacript.
It should print out "ok" if the button was clicked or "error" if it times out.

Notice to avoid the race condition of the button being clicked before we start listening to
it, we add a buffer of 1 to the channel.

We also create a transient div, and append the button there -- making the button not appear after it is saved.
No real need for that, other than to also test the `AppendTo` method.

In [4]:
import (
    "github.com/janpfeifer/gonb/gonbui"
    "github.com/janpfeifer/gonb/gonbui/dom"
    "github.com/janpfeifer/gonb/gonbui/widgets"
)

%%
htmlId := "gonb_slider_" + gonbui.UniqueId()  // Not needed, just so we test HtmlId() method below.
rootId := dom.CreateTransientDiv()  // Not needed, just so we test AppentTo() method below.
button := widgets.Button("Ok").HtmlId(htmlId).AppendTo(rootId).Done()
buttonChannel := button.Listen().WithBuffer(1)
if button.GetHtmlId() != htmlId {
    fmt.Println("error: wrong htmlId")
    return
}

// Click on button with Javascript after 100ms.
go func() {
    time.Sleep(100 * time.Millisecond);
    dom.TransientJavascript(fmt.Sprintf(`
(() => {
    let button = document.getElementById("%s");
    button.click();
    console.log("Clicked!");
})();
`, button.GetHtmlId()))
}()

// Wait for button click or timeout.
select {
case <-buttonChannel.C:
    fmt.Println("clicked")
case <-time.After(500 * time.Millisecond):
    fmt.Println("error")
}
buttonChannel.Close()


clicked


### Sliders

We create a slider, and then interact with it with Javascript and test we get the correct reading.

In [5]:
%%
const answer = 42

htmlId := "gonb_slider_" + gonbui.UniqueId()  // Not needed, just so we test HtmlId() method below.
rootId := dom.CreateTransientDiv()  // Not needed, just so we test AppentTo() method below.
slider := widgets.Slider(0, 100, 50).HtmlId(htmlId).AppendTo(rootId).Done()
sliderChan := slider.Listen().WithBuffer(1)
if slider.GetHtmlId() != htmlId {
    fmt.Println("error: wrong htmlId")
    return
}

go func() {
    time.Sleep(100 * time.Millisecond);
    dom.TransientJavascript(fmt.Sprintf(`
(() => {
    let slider = document.getElementById("%s");
    const answer = %d;
    slider.value = answer;  // Doesn't trigger "change" event, unfortunately.
    slider.dispatchEvent(new Event("change"));
    console.log("Slider set to "+answer);
})();
`, slider.GetHtmlId(), answer))
}()

// Wait for slider change or timeout.
select {
case newValue := <-sliderChan.C:
    time.Sleep(100 * time.Millisecond)  // Make sure slider also received the update.
    // Checks we receive the new value, and that the slider object also recorded it.
    if newValue != answer || slider.GetValue() != answer {
        fmt.Printf("error: received %d, slider registered %d", newValue, slider.GetValue())
    } else  {
        fmt.Println("slider correctly set")
    }
case <-time.After(500000 * time.Millisecond):
    fmt.Println("error: timedout")
}

slider correctly set
